In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
savePath = '../input/newSplited/'

In [3]:
train_df = pd.read_parquet('../input/splited/train.parquet')
train_df

,session,aid,ts,type
0,0,1517085,1659304800,0
1,0,1563459,1659304904,0
2,0,1309446,1659367439,0
3,0,16246,1659367719,0
4,0,1781822,1659367871,0
...,...,...,...,...
163955175,11098523,175715,1661119197,0
163955176,11098524,1088524,1661119198,0
163955177,11098525,182927,1661119199,0
163955178,11098526,510055,1661119199,0


In [4]:
WEEK = 1
second = WEEK * 7*24*60*60

In [5]:
valIdx = train_df.groupby('session').agg({'ts': 'min'}).ts > 1661119199 - second
valIdx

session
0           False
1           False
2           False
3           False
4           False
            ...  
11098523     True
11098524     True
11098525     True
11098526     True
11098527     True
Name: ts, Length: 11098528, dtype: bool

In [6]:
train_df = train_df.merge(valIdx, on='session', how='left')
train_df

,session,aid,ts_x,type,ts_y
0,0,1517085,1659304800,0,False
1,0,1563459,1659304904,0,False
2,0,1309446,1659367439,0,False
3,0,16246,1659367719,0,False
4,0,1781822,1659367871,0,False
...,...,...,...,...,...
163955175,11098523,175715,1661119197,0,True
163955176,11098524,1088524,1661119198,0,True
163955177,11098525,182927,1661119199,0,True
163955178,11098526,510055,1661119199,0,True


## newTrain

In [7]:
newTrain_df = train_df[train_df.ts_y == False].drop(columns='ts_y').reset_index(drop=True)
newTrain_df.columns = ['session', 'aid', 'ts', 'type']

newTrain_df

,session,aid,ts,type
0,0,1517085,1659304800,0
1,0,1563459,1659304904,0
2,0,1309446,1659367439,0
3,0,16246,1659367719,0
4,0,1781822,1659367871,0
...,...,...,...,...
144332289,8643218,12236,1660514471,0
144332290,8643218,1072581,1660514631,0
144332291,8643218,668368,1660514674,0
144332292,8643219,1112446,1660514399,0


In [49]:
newTrain_df.to_parquet(savePath + 'train.parquet')

## Val

In [8]:
val_df = train_df[train_df.ts_y].drop(columns='ts_y').reset_index(drop=True)
val_df.columns = ['session', 'aid', 'ts', 'type']
val_df

,session,aid,ts,type
0,8643220,573273,1660514400,0
1,8643220,1189975,1660514432,0
2,8643221,921137,1660514400,0
3,8643221,111691,1660514932,0
4,8643222,930597,1660514400,0
...,...,...,...,...
19622881,11098523,175715,1661119197,0
19622882,11098524,1088524,1661119198,0
19622883,11098525,182927,1661119199,0
19622884,11098526,510055,1661119199,0


In [9]:
val_df = val_df.groupby('session').filter(lambda x : len(x)>1)
val_df

,session,aid,ts,type
0,8643220,573273,1660514400,0
1,8643220,1189975,1660514432,0
2,8643221,921137,1660514400,0
3,8643221,111691,1660514932,0
4,8643222,930597,1660514400,0
...,...,...,...,...
19622839,11098483,46115,1661119194,0
19622850,11098494,1733630,1661119183,0
19622851,11098494,1733630,1661119199,0
19622866,11098509,159067,1661119189,0


In [10]:
isSelect = val_df.groupby(['session']).apply(lambda x: x.index > random.sample(list(x.index)[:-1], 1)[0]).explode()
val_df['B'] = isSelect.values
val_df

,session,aid,ts,type,B
0,8643220,573273,1660514400,0,False
1,8643220,1189975,1660514432,0,True
2,8643221,921137,1660514400,0,False
3,8643221,111691,1660514932,0,True
4,8643222,930597,1660514400,0,False
...,...,...,...,...,...
19622839,11098483,46115,1661119194,0,True
19622850,11098494,1733630,1661119183,0,False
19622851,11098494,1733630,1661119199,0,True
19622866,11098509,159067,1661119189,0,False


### Val A

In [11]:
val_A = val_df[val_df['B'] == False]
val_A = val_A.drop(columns='B').reset_index(drop=True)
val_A

,session,aid,ts,type
0,8643220,573273,1660514400,0
1,8643221,921137,1660514400,0
2,8643222,930597,1660514400,0
3,8643222,1037630,1660514457,0
4,8643222,756920,1660514485,0
...,...,...,...,...
9699763,11098472,613059,1661119170,0
9699764,11098478,261405,1661119174,0
9699765,11098483,38826,1661119179,0
9699766,11098494,1733630,1661119183,0


In [65]:
val_A.to_parquet(savePath + 'val_A.parquet')

### Val B

In [12]:
val_B = val_df[val_df['B'] == True]
val_B = val_B.drop(columns='B')
val_B

,session,aid,ts,type
1,8643220,1189975,1660514432,0
3,8643221,111691,1660514932,0
8,8643222,756920,1660514636,0
9,8643222,1410436,1660920541,0
10,8643222,1410436,1660944638,0
...,...,...,...,...
19622826,11098472,504876,1661119191,0
19622833,11098478,261405,1661119190,0
19622839,11098483,46115,1661119194,0
19622851,11098494,1733630,1661119199,0


In [13]:
val_B = val_B.groupby(['session', 'type']).aid.apply(lambda x: list(x))
val_B = val_B.to_frame()
val_B.reset_index(inplace=True)
val_B = val_B.rename(columns = {'aid':'ground_truth'})
val_B

,session,type,ground_truth
0,8643220,0,[1189975]
1,8643221,0,[111691]
2,8643222,0,"[756920, 1410436, 1410436, 1104129, 1568399, 1..."
3,8643223,0,"[1811963, 1742315]"
4,8643224,0,"[933835, 1502227, 1706288, 1106262, 778561, 77..."
...,...,...,...
2756237,11098472,0,[504876]
2756238,11098478,0,[261405]
2756239,11098483,0,[46115]
2756240,11098494,0,[1733630]


In [14]:
types = {0: 'clicks', 1: 'carts', 2: 'orders'}
val_B["type"] = val_B["type"].map(types)
val_B

,session,type,ground_truth
0,8643220,clicks,[1189975]
1,8643221,clicks,[111691]
2,8643222,clicks,"[756920, 1410436, 1410436, 1104129, 1568399, 1..."
3,8643223,clicks,"[1811963, 1742315]"
4,8643224,clicks,"[933835, 1502227, 1706288, 1106262, 778561, 77..."
...,...,...,...
2756237,11098472,clicks,[504876]
2756238,11098478,clicks,[261405]
2756239,11098483,clicks,[46115]
2756240,11098494,clicks,[1733630]


In [68]:
val_B.to_parquet(savePath + 'val_B.parquet')